<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/ibd_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Screening Using IBD's RS Rating Methodology

This notebook demonstrates stock screening techniques inspired by the methodology used in Investor's Business Daily (IBD). It incorporates the (price) RS Rating, EPS RS Rating, and Revenue RS Rating to evaluate stocks based on price performance, earnings, and revenue growth. The Price RS Rating typically reflects a stock’s performance over the past year, but a 3-month version is also available to assist with short-term trading decisions.

## Install and Setup (this section will be executed automatically)

### Install Required Packages

In [ ]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ranking_utils"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ranking_utils) to /tmp/pip-req-build-slfdbghr
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-slfdbghr
  Running command git checkout -b feature/ranking_utils --track origin/feature/ranking_utils
  Switched to a new branch 'feature/ranking_utils'
  Branch 'feature/ranking_utils' set up to track remote branch 'feature/ranking_utils' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit f7f08948581269857ec6cff2f9e27922231b299d
  Preparing metadata (setup.py) ... done
--2024-10-11 12:31:23--  https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0
Resolving share.cole.tw (share.cole.tw)... 180.218.0.193
Connecting to share.cole.tw (share.cole.tw)|180.218.0.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7110560 (6.8M) [

### Setup and Configuration

In [ ]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [ ]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [ ]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        if dir_path:
            url = f'{self.api_url}/{dir_path}?ref={self.branch}'
        else:
            url = f'{self.api_url}?ref={self.branch}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        elif response.status_code == 404:
            print(f"Directory not found: {url}")
            return []
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    dir='ibd',
    branch='data'
)

In [ ]:
# @title Taiwan Stock Name Lookup

class StockNameLookup:
    _df = None  # Class-level variable to hold the DataFrame

    @classmethod
    def _load_data(cls):
        if cls._df is None:  # Check if the DataFrame is already loaded
            gh = ReadOnlyGitHub(
                repo_owner='yorkjong',
                repo_name='stock-reports',
                dir='data/stock_list',
            )
            cls._df = gh.download_csv('taiwan_stock_OpenAPI.csv')

    @classmethod
    def tw_stock_name(cls, ticker):
        cls._load_data()  # Ensure data is loaded before accessing

        code = ticker.split('.')[0]  # Extract the code part

        # Filter the DataFrame to find the stock name for the given code
        stock_name = cls._df.loc[cls._df['Code'] == code, 'Name']

        # Check if the stock_name is empty and return an appropriate message
        if not stock_name.empty:
            return stock_name.values[0]  # Return the first matched stock name
        else:
            return None  # Return None if ticker not found


def tw_stock_name(ticker):
    return StockNameLookup.tw_stock_name(ticker)

In [ ]:
# @title Metadata

def parse_metadata(filename):
    components = filename.split('_')
    if len(components) != 5:
        raise ValueError("Filename does not have the expected number of components")

    source, kind, period, rs_win, date = components
    rs_win = rs_win.replace('ibd', '')
    date = date.replace('.csv', '')

    return {
        "Source": source,
        "Type": kind,
        "Period": period,
        "RS window": rs_win,
        "Date": date
    }


def print_metadata(meatadata):
    for key, value in meatadata.items():
        print(f"{key}: {value}")

In [ ]:
# @title IBD Financial files
import re
from datetime import datetime


def get_latest_file(file_list, source):
    pattern = rf'^{re.escape(source)}_(\w+)_fin_(\d+)\.csv$'
    matching_files = []

    for file in file_list:
        match = re.match(pattern, file)
        if match:
            date_str = match.group(2)
            date = datetime.strptime(date_str, "%Y%m%d")
            matching_files.append((file, date))

    if not matching_files:
        return None

    return max(matching_files, key=lambda x: x[1])[0]


def fin_download_latest_csv(source):
    gh = ReadOnlyGitHub(
        repo_owner='yorkjong',
        repo_name='stock-reports',
        dir='ibd_fin',
        branch='data'
    )
    file_list = gh.list_filenames()
    fin_filename = get_latest_file(file_list, source)
    df = gh.download_csv(fin_filename)
    return df, fin_filename

In [ ]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [ ]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return [name.strip() for names in df['Name']
                for name in names.split(',')]
    elif 'Ticker' in df.columns:
        return [ticker.strip() for tickers in df['Ticker']
                for ticker in tickers.split(',')]
    return []

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [ ]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [ ]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_rs_window_dropdown(value='3mo'):
    return widgets.Dropdown(
        options=['3mo', '12mo'],
        value=value,
        description='rs_window:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [ ]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [ ]:
# @title Outputs
outputs = widgets.VBox()

In [ ]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The source of stocks to analyze)
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector:
  - For Taiwan Markets, possible values include:
    - **`TWSE`**: Taiwan Stock Exchange (台灣上市股票交易所)
    - **`TPEX`**: Taipei Exchange (上櫃交易所)
    - **`ESB`**: Emerging Stock Board (興櫃交易所)
  - Can also be combined with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`).
  - For America Markets, possible values include:
    - **`SPX`**: S&P 500 (標普五百指數)
    - **`DJIA`**: Dow Jones Industrial Average (道瓊指數)
    - **`NDX`**: NASDAQ-100 (納斯達克一百指數)
    - **`SOX`**: PHLX Semiconductor Index (費半指數)
  - Multiple indices can be combined using `+` (e.g., `SPX+DJIA+NDX+SOX`).

#### Period (Historical Data Time Range)
- The time range for which to fetch historical data.
  - **`2y`**: 2 years
  - **`6mo`**: 6 months

#### RS Window (Window for RS Calculation)
- The time window ('3mo' or '12mo') for Relative Strength:
  - **`3mo`**: 3 months
  - **`12mo`**: 12 months

#### RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
  - A value of 100 represents the performance of the benchmark index or market.
- The IBD RS calculates the performance over the last year, with the most recent quarter weighted double.
- The IBD 3-month RS calculates the performance over the last quarter.

#### RS Rating
- RS Rating, ranging from 1 (worst) to 99 (best), evaluates the price performance of a stock relative to a benchmark index.

# Execute Actions Step by Step


## Step 1. Select and Preview a File

---



In [47]:
# @title Step 1.1 Select a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
#all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
#all_periods = sorted(set(fn.split('_')[2] for fn in filenames))
all_rs_windows = sorted(set(fn.split('_')[3].replace('ibd', '') for fn in filenames))

# Calculate the maximum length of filenames to set dropdown width
if filenames:
    max_filename_length = max(len(fn) for fn in filenames)
else:
    max_filename_length = 0
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

rs_window_dropdown = widgets.Dropdown(
    options=all_rs_windows,
    description='RS window:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

output = widgets.Output()

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update RS window dropdown
    available_rs_windows = sorted(set(fn.split('_')[3].replace('ibd', '') for fn in date_filtered_files))
    rs_window_dropdown.options = available_rs_windows
    if rs_window_dropdown.value not in available_rs_windows:
        rs_window_dropdown.value = available_rs_windows[0] if available_rs_windows else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    # Filter files based on selected date, source, and RS window
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            rs_window_dropdown.value == fn.split('_')[3].replace('ibd', ''))
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        global selected_file, metadata
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
        selected_file = file_dropdown.value
        metadata = parse_metadata(selected_file)
        with output:
            output.clear_output()
            print_metadata(metadata)
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
rs_window_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, rs_window_dropdown, file_dropdown, output)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='410px'), options=('20241012',), value='20241012')

Dropdown(description='Source:', layout=Layout(width='410px'), options=('TWSE+TPEX', 'U.S.Listed'), value='TWSE…

Dropdown(description='RS window:', layout=Layout(width='410px'), options=('12mo', '3mo'), value='12mo')

Dropdown(description='File:', layout=Layout(width='410px'), options=(), value=None)

Output()

In [65]:
# @title Step 1.2 Load and Preview the Files
print(f'{selected_file}:')
with requests_cache.disabled():
    df_rs = github.download_csv(selected_file)
df_rs = df_rs.drop(['1wk:end max',
                    'Rating (3M:1M)', 'Rating (6M:3M)', 'Rating (9M:6M)'],
                   axis=1)
display(df_rs)

with requests_cache.disabled():
    df_fin, fn_fin = fin_download_latest_csv(metadata['Source'])
print(f'\n{fn_fin}:')
columns_to_keep = ['Ticker', 'EPS RS', 'TTM EPS', 'Rev RS', 'TTM RPS', 'TTM PE',
                   'Rating (EPS RS)', 'Rating (Rev RS)']
df_fin = df_fin[columns_to_keep]
display(df_fin)

print(f'\nMerged:')
df_merge = pd.merge(df_rs, df_fin, on='Ticker', how='left')
display(df_merge)

U.S.Listed_stocks_1y_ibd3mo_20241012.csv:


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,52W pos,MA50,MA200,Volume / VMA50
0,TECX,Healthcare,Biotechnology,17603.77,229254.00,160923.38,1778.56,86.87,99,31.36,0.95,21.27,17.29,0.92
1,MIRA,Healthcare,Drug Manufacturers—General,7017.33,6157.41,3316.67,103.03,90.71,99,1.09,0.12,1.57,1.17,0.17
2,ASNS,Technology,Communication Equipment,986.13,8637.73,17037.48,1591.44,101.27,99,1.34,0.28,1.48,1.23,0.23
3,RGC,Healthcare,Drug Manufacturers—Specialty & Generic,966.89,882.84,601.35,89.32,70.98,99,6.26,0.21,7.04,6.29,0.69
4,GNLN,Consumer Defensive,Tobacco,760.81,642.17,314.95,126.01,88.43,99,4.15,0.14,5.75,5.34,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,XPON,Industrials,Electrical Equipment & Parts,14.53,17.19,52.09,67.40,102.63,1,2.39,0.00,7.75,200.98,2.41
5430,NDRA,Healthcare,Diagnostics & Research,13.86,23.13,40.99,69.04,127.32,1,0.21,0.00,0.91,23.21,0.17
5431,CNSP,Healthcare,Biotechnology,10.33,9.56,27.75,55.19,123.57,1,0.12,0.00,0.15,11.29,0.19
5432,SYTA,Technology,Communication Equipment,9.61,13.03,53.18,82.35,74.50,1,0.84,0.00,1.62,35.24,0.05



U.S.Listed_stocks_fin_20241010.csv:


,Ticker,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
0,LUNR,6.003049e+09,-0.17,83.11,4.810,NaN,99.0,91.0
1,BZFD,4.060600e+09,-1.12,-36.62,6.229,NaN,99.0,9.0
2,CART,2.300000e+09,-5.56,7.22,13.986,NaN,99.0,76.0
3,ACDC,1.882714e+09,-1.43,5.76,13.919,NaN,99.0,74.0
4,KNTK,1.644445e+09,2.83,18.60,24.316,17.17,99.0,83.0
...,...,...,...,...,...,...,...,...
5437,ZJYL,NaN,0.02,NaN,0.129,127.00,NaN,NaN
5438,ZK,NaN,-4.99,NaN,314.047,NaN,NaN,NaN
5439,ZKIN,NaN,-1.95,NaN,3.592,NaN,NaN,NaN
5440,ZOOZ,NaN,-22.44,NaN,0.069,NaN,NaN,NaN



Merged:


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,...,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
0,TECX,Healthcare,Biotechnology,17603.77,229254.00,160923.38,1778.56,86.87,99,31.36,...,21.27,17.29,0.92,NaN,-3.07,NaN,NaN,NaN,NaN,NaN
1,MIRA,Healthcare,Drug Manufacturers—General,7017.33,6157.41,3316.67,103.03,90.71,99,1.09,...,1.57,1.17,0.17,126.96,-0.86,-1.191000e+01,NaN,NaN,25.0,30.0
2,ASNS,Technology,Communication Equipment,986.13,8637.73,17037.48,1591.44,101.27,99,1.34,...,1.48,1.23,0.23,2352.19,-1.40,3.606000e+01,1.772,NaN,96.0,88.0
3,RGC,Healthcare,Drug Manufacturers—Specialty & Generic,966.89,882.84,601.35,89.32,70.98,99,6.26,...,7.04,6.29,0.69,NaN,-0.36,NaN,NaN,NaN,NaN,NaN
4,GNLN,Consumer Defensive,Tobacco,760.81,642.17,314.95,126.01,88.43,99,4.15,...,5.75,5.34,0.01,2176.56,18.78,-4.508900e+02,61.566,0.23,96.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,XPON,Industrials,Electrical Equipment & Parts,14.53,17.19,52.09,67.40,102.63,1,2.39,...,7.75,200.98,2.41,100.69,-119.00,-1.009000e+01,70.928,NaN,22.0,40.0
5430,NDRA,Healthcare,Diagnostics & Research,13.86,23.13,40.99,69.04,127.32,1,0.21,...,0.91,23.21,0.17,441.12,-32.00,-1.191000e+01,NaN,NaN,85.0,30.0
5431,CNSP,Healthcare,Biotechnology,10.33,9.56,27.75,55.19,123.57,1,0.12,...,0.15,11.29,0.19,580.15,-367.38,-1.191000e+01,NaN,NaN,89.0,30.0
5432,SYTA,Technology,Communication Equipment,9.61,13.03,53.18,82.35,74.50,1,0.84,...,1.62,35.24,0.05,381.16,295.46,-3.674000e+01,175.111,0.00,82.0,9.0


## Step 2. Filter Stocks

In [66]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond2 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond3 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond4 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond5 = "& (df['TTM EPS'] > 0)" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond6 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS" # @param ["RS","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS", "Rev RS"]
ascending = False # @param {"type":"boolean"}
num_items = 30 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 7
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241012


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,...,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
65,FFIE,Consumer Cyclical,Auto Manufacturers,202.46,934.05,10380.71,2852.55,46.24,98,2.51,...,4.99,12.07,0.44,5302.38,866.40,2.824444e+15,0.378,0.00,98.0,97.0
70,SEZL,Financial Services,Credit Services,198.84,198.40,202.61,416.19,413.67,98,179.75,...,138.71,80.75,0.86,1202.40,7.05,4.517000e+01,33.982,22.63,94.0,89.0
97,ADMA,Healthcare,Biotechnology,175.72,178.91,179.77,160.59,133.68,97,17.15,...,17.60,10.23,1.60,514.34,0.14,6.802000e+01,1.447,121.50,87.0,90.0
116,CMPO,Industrials,Metal Fabrication,168.00,164.43,141.16,122.76,101.38,97,14.30,...,12.02,7.72,0.55,158.64,1.04,1.400000e-01,19.384,13.37,34.0,66.0
125,JVA,Consumer Defensive,Packaged Foods,163.48,166.44,152.80,145.47,141.37,97,3.14,...,2.55,1.77,0.68,1004.75,0.20,1.350000e+00,13.333,14.64,93.0,68.0
130,BYU,Financial Services,Shell Companies,160.82,167.21,148.93,85.07,74.44,97,0.30,...,1.72,1.40,7.26,695.39,1.68,3.079000e+01,13.657,0.28,91.0,87.0
159,LWAY,Consumer Defensive,Packaged Foods,150.41,138.04,109.83,154.44,126.86,96,26.63,...,20.55,16.37,0.35,342.89,0.90,1.037000e+01,12.028,29.56,80.0,78.0
165,YOU,Technology,Software—Application,148.48,144.94,129.23,90.71,116.01,96,34.18,...,29.52,21.77,0.70,510.41,0.78,1.811000e+01,7.586,42.67,87.0,83.0


FFIE, SEZL, ADMA, CMPO, JVA, BYU, LWAY, YOU


In [67]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "" # @param ["& (df['RS'] > df['1wk:end max'])", ""]
cond2 = "" # @param ["& (df['1wk:end max'] > df['1mo:1wk max'])", ""]
cond3 = "& (df['1mo:1wk max'] > df['3mo:1mo max'])" # @param ["& (df['1mo:1wk max'] > df['3mo:1mo max'])", ""]
cond4 = "& (df['3mo:1mo max'] > df['6mo:3mo max'])" # @param ["& (df['3mo:1mo max'] > df['6mo:3mo max'])", ""]
cond5 = "& (df['6mo:3mo max'] > df['9mo:6mo max'])" # @param ["& (df['6mo:3mo max'] > df['9mo:6mo max'])", ""]
cond6 = "& (df['Rating (RS)'] > 90)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond8 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond9 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond10 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond11 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 20 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 12
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241012


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,...,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
116,CMPO,Industrials,Metal Fabrication,168.00,164.43,141.16,122.76,101.38,97,14.30,...,12.02,7.72,0.55,158.64,1.04,0.14,19.384,13.37,34.0,66.0
125,JVA,Consumer Defensive,Packaged Foods,163.48,166.44,152.80,145.47,141.37,97,3.14,...,2.55,1.77,0.68,1004.75,0.20,1.35,13.333,14.64,93.0,68.0
130,BYU,Financial Services,Shell Companies,160.82,167.21,148.93,85.07,74.44,97,0.30,...,1.72,1.40,7.26,695.39,1.68,30.79,13.657,0.28,91.0,87.0
241,SEI,Energy,Oil & Gas Equipment & Services,137.96,140.57,128.10,105.68,89.55,95,13.96,...,12.39,9.68,0.55,625.00,0.66,1.36,9.563,20.55,90.0,68.0
254,PNTG,Healthcare,Medical Care Facilities,136.44,137.55,131.27,130.56,124.19,94,33.79,...,32.93,23.58,0.68,234.49,0.64,10.39,20.380,51.47,68.0,78.0
336,VIRT,Financial Services,Capital Markets,129.70,127.81,122.90,114.06,100.61,93,32.55,...,30.60,23.50,0.50,382.83,1.99,2.82,20.318,16.15,82.0,70.0
353,HNNA,Financial Services,Asset Management,128.77,128.11,120.07,99.83,95.86,93,9.61,...,9.73,7.75,0.53,198.05,0.80,8.47,3.543,12.09,57.0,77.0
364,KARO,Technology,Software—Application,128.17,128.13,127.05,114.15,99.11,92,43.70,...,38.08,30.39,2.30,206.83,1.49,1.32,138.674,26.68,60.0,68.0
424,PAYO,Technology,Software—Infrastructure,125.43,122.27,112.86,108.41,88.72,91,7.89,...,7.19,5.69,0.52,327.18,0.26,10.07,2.511,30.29,79.0,78.0


CMPO, JVA, BYU, SEI, PNTG, VIRT, HNNA, KARO, PAYO


In [68]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 100 # @param [100, 130, 160, 200] {"type":"raw"}
cond0 = "(df['RS'] > base)" # @param ["(df['RS'] > base)"]
cond1 = "" # @param ["& (df['1wk:end max'] < base)",""]
cond2 = "" # @param ["& (df['1mo:1wk max'] < base)",""]
cond3 = "& (df['3mo:1mo max'] < base)" # @param ["& (df['3mo:1mo max'] < base)",""]
cond4 = "& (df['6mo:3mo max'] < base)" # @param ["& (df['6mo:3mo max'] < base)",""]
cond5 = "& (df['9mo:6mo max'] < base)" # @param ["& (df['9mo:6mo max'] < base)",""]
cond6 = "" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& (df['Volume / VMA50'] > 2)" # @param ["& (df['Volume / VMA50'] > 1.5)", "& (df['Volume / VMA50'] > 2)", "& (df['Volume / VMA50'] > 3)", "& (df['Volume / VMA50'] > 4)", ""]
cond8 = "& (df['Price'] > df['MA200'])" # @param ["& (df['Price'] > df['MA50'])", "& (df['Price'] > df['MA200'])", ""]
cond9 = "& (df['MA50'] > df['MA200'])" # @param ["& (df['MA50'] > df['MA200'])",""]
cond10 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond11 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond12 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond13 = "" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond14 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 15
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241012


,Ticker,Sector,Industry,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Price,...,MA50,MA200,Volume / VMA50,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)


In [52]:
# @title Filter 4. Groupby "Industry" {"run":"auto"}
cond0 = "(df['RS'] > 100)" # @param ["(df['RS'] > 100)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
num_items = 2 # @param [1, 2, 3, 4, 5] {"type":"raw"}

NUM_CONDS = 2

from vistock.ranking_utils import append_ratings, groupby_industry

print_metadata(metadata)
stock_df = df_merge.copy()

# Filter out rows with NaN in the 'Ticker' column
#stock_df = stock_df[stock_df['Ticker'].notna()]

rs_columns = ['RS',
              '1mo:1wk max', '3mo:1mo max', '6mo:3mo max', '9mo:6mo max']
if 'Name' in stock_df.columns:
    columns = ['Sector', 'Ticker', 'Name'] + rs_columns
else:
    columns = ['Sector', 'Ticker'] + rs_columns
df = groupby_industry(stock_df, columns, key='RS')

df = df.sort_values(by='RS', ascending=False).reset_index(drop=True)
rating_columns = ['Rating (RS)', 'Rating (1M:1W)', 'Rating (3M:1M)',
                  'Rating (6M:3M)', 'Rating (9M:6M)']
df = append_ratings(df, rs_columns, rating_columns)

cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))
df = df[eval(cond)]
df_top_f4 = df.head(num_items)
display(df_top_f4)

print_column(df_top_f4, 'Name')
print_column(df_top_f4, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 1y
RS window: 3mo
Date: 20241012


,Industry,Sector,Ticker,RS,1mo:1wk max,3mo:1mo max,6mo:3mo max,9mo:6mo max,Rating (RS),Rating (1M:1W),Rating (3M:1M),Rating (6M:3M),Rating (9M:6M)
0,Drug Manufacturers—General,Healthcare,"MIRA,GRFS,GILD,BMY,ABBV,SNY,NVS,LLY,JNJ,AMGN,A...",484.18,440.63,283.92,101.86,101.94,99,98,96,24,19
1,Tobacco,Consumer Defensive,"GNLN,KAVL,TPB,PM,BTI,MO,UVV,ISPR,RLX,XXII",175.94,185.95,164.06,142.01,94.24,98,96,94,92,4


MIRA,GRFS,GILD,BMY,ABBV,SNY,NVS,LLY,JNJ,AMGN,AZN,PFE,GSK,OGN,MRK,BIIB,SCLX,AMRN, GNLN,KAVL,TPB,PM,BTI,MO,UVV,ISPR,RLX,XXII


## Step 3. Visualize Filtered Stocks

In [ ]:
# @title Plot 1. IBD Relative Strength Comparison {"run":"auto"}
source = "Filter 4. Groupby Industry" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout", "Filter 4. Groupby Industry", "Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown(metadata['Period'])
interval_dropown = create_interval_dropdown('1d')
rs_window_dropdown = create_rs_window_dropdown(metadata['RS window'])

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown, rs_window_dropdown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

rs_cmp = {
    'mplfinance': mpl.ibd_rs_cmp,
    'Plotly': ply.ibd_rs_cmp,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        rs_window = rs_window_dropdown.value
        if rs_cmp is mpl.ibd_rs_cmp:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        rs_window=rs_window,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [ ]:
# @title Plot 2. IBD Stock Chart {"run":"auto"}
source = "Filter 4. Groupby Industry" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Filter 4. Groupby Industry","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw_stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown(metadata['Period'])
interval_dropown2 = create_interval_dropdown('1d')
rs_window_dropdown2 = create_rs_window_dropdown(metadata['RS window'])

stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, rs_window_dropdown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl as mpl
import vistock.plotly as ply

stock_chart = {
    'mplfinance': mpl.ibd_rs,
    'Plotly': ply.ibd_rs,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        rs_window = rs_window_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl.ibd_rs:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             rs_window=rs_window,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()
